In [1]:
import boto3
import pandas as pd
import os.path
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [74]:
s3 = boto3.resource('s3')
client = boto3.client("s3")

bucket = "ehrlich-arb-data"
dateid = "20171210"

buck = s3.Bucket(bucket)

s3files = []
for o in buck.objects.filter(Prefix=dateid).all():
    s3files.append(o.key)

print("%s files found in %s" % (len(s3files), bucket))

54 files found in ehrlich-arb-data


In [73]:
if os.path.isdir(dateid) == False:
    os.mkdir(dateid)

for f in s3files:
    if os.path.isfile(f) == False:
        print("Downloading %s ... " % (f), end='')
        client.download_file(bucket, f, f)
        print("done")

In [69]:
files = []
for file in os.listdir(dateid):
    files.append(dateid + "/" + file)
    print(file)

ls.s3.f0647fa7-5617-4712-836c-096f38c5dd5b.2017-12-08T21.33.part50.txt.gz
ls.s3.e519401b-4267-4e62-a7ce-227a91cb8c35.2017-12-08T13.04.part30.txt.gz
ls.s3.61a60781-4fa7-4ad2-a9ef-9d0dc89d7f45.2017-12-08T16.04.part37.txt.gz
ls.s3.96d49a58-cdaf-47f0-b50b-e3dbcbc900cb.2017-12-08T03.04.part7.txt.gz
ls.s3.72571220-5195-4633-8a8a-1dce29ede4cb.2017-12-08T17.20.part40.txt.gz
ls.s3.49d4050a-b05d-48bb-9aab-955507f0f748.2017-12-08T08.48.part20.txt.gz
ls.s3.32200de4-9f46-44c1-a7c4-aba3364f53e6.2017-12-08T21.58.part51.txt.gz
ls.s3.32ae137a-e709-443f-a755-7ae9bb3d0ba1.2017-12-08T05.17.part12.txt.gz
ls.s3.c3a6d82f-97f4-45c7-bf2b-0929846067a0.2017-12-08T02.38.part6.txt.gz
ls.s3.63d1050e-167a-4550-b256-29a904186145.2017-12-08T20.42.part48.txt.gz
ls.s3.1dafe616-08f7-4100-9e7e-bf401eb00b0f.2017-12-08T13.55.part32.txt.gz
ls.s3.642827a5-f031-48f1-9108-166edb5b67b9.2017-12-08T12.13.part28.txt.gz
ls.s3.01c6e4c3-3057-4933-8ea6-5d5fb2e3e506.2017-12-08T16.29.part38.txt.gz
ls.s3.d70c3f94-c10a-4f2c-a8d0-2b6c0cdbbf

In [71]:
l = []
d = {'ts': float, 'exchange': str, 'tradingPair': str, 'orderType': str, 'price': float, 'amount': float}
n = ["ts", 'exchange', 'tradingPair', 'orderType', 'price', 'amount']
for (i, f) in enumerate(files):
    df = pd.read_csv(f, dtype = d, header=None, names = n, error_bad_lines = False, engine = 'python', na_values = ["Connection aborted"])
    l.append(df)

IsADirectoryError: [Errno 21] Is a directory: '20171208/tmp'

In [64]:
data = pd.concat(l)

In [65]:
data.head()

,ts,exchange,tradingPair,orderType,price,amount
0,1.512865e+12,BITTREX,ARKBTC,SELL,0.000431,10.140947
1,1.512865e+12,COINGI,LTCUSD,SELL,150.000000,2.450000
2,1.512865e+12,COINGI,LTCUSD,SELL,152.650000,2.151000
3,1.512865e+12,COINGI,LTCUSD,SELL,156.500000,2.540000
4,1.512865e+12,COINGI,LTCUSD,SELL,162.150000,2.185000


In [49]:
data.where(data.exchange == "COINGI").where(data.tradingPair == "LTCBTC").where(data.orderType == "BUY").price.describe()

count    6.337566e+06
mean     2.991751e-03
std      1.649658e-03
min      8.000000e-04
25%      1.700000e-03
50%      2.600000e-03
75%      4.190000e-03
max      9.000000e-03
Name: price, dtype: float64

In [25]:
data.where(data.exchange == "BITTREX").where(data.tradingPair == "LTCBTC").where(data.orderType == "BUY").price.describe()

count    23707.000000
mean         0.038857
std          0.012587
min          0.000479
25%          0.036331
50%          0.044382
75%          0.047639
max          0.049350
Name: price, dtype: float64

In [66]:
data.groupby([data.tradingPair, data.exchange]).count()

ts  exchange  tradingPair  orderType   price  amount
tradingPair exchange                                                          
ARKBTC      BITTREX     1050      1050         1050       1050    1050    1050
BTCEUR      COINGI    185707    185707       185707     185707  185707  185707
BTCUSD      COINGI    289860    289860       289860     289860  289860  289860
DASHBTC     BITTREX    30652     30652        30652      30652   30652   30652
            COINGI    120045    120045       120045     120045  120045  120045
LTCBTC      COINGI    192525    192525       192525     192525  192525  192525
LTCUSD      COINGI    194790    194790       194790     194790  194790  194790
MCOBTC      BITTREX      198       198          198        198     198     198
PPCBTC      COINGI    165345    165345       165345     165345  165345  165345
PPCUSD      COINGI    101925    101925       101925     101925  101925  101925
VTCBTC      BITTREX     1579      1579         1579       1579    1579    1579
            COINGI    176670    176670       176670     176670  176670  176670

In [51]:
data.where(data.tradingPair == "PPCBTC").where(data.exchange == "BITTREX").count()

ts             0
exchange       0
tradingPair    0
orderType      0
price          0
amount         0
dtype: int64

In [42]:
data.where(data.exchange == "BITTREX").groupby(data.tradingPair).count()

,ts,exchange,tradingPair,orderType,price,amount
tradingPair,,,,,,
BTCEUR,0,0,0,0,0,0
BTCUSD,0,0,0,0,0,0
DASHBTC,55767,55767,55767,55767,55767,55767
LTCBTC,0,0,0,0,0,0
LTCUSD,0,0,0,0,0,0
PPCBTC,0,0,0,0,0,0
PPCUSD,0,0,0,0,0,0
VTCBTC,0,0,0,0,0,0


In [12]:
data['dt'] = pd.to_datetime(data['ts']/1000,unit='s')
data.where(data.tradingPair == "PPCBTC").where(data.orderType == "BUY")[['dt', 'price']].plot(figsize=(10, 8))

KeyboardInterrupt: 